## Using macros

In [1]:
@time 1 + 2

  0.000001 seconds


3

In [6]:
@time (1 + 2)

  0.000001 seconds


3

In [8]:
@assert 2 == 4 "2 is not equal 4"

LoadError: AssertionError: 2 is not equal 4

In [9]:
@assert( 2 == 4, "2 is not equal 4")

LoadError: AssertionError: 2 is not equal 4

Note that if you do not use parentheses, the expressions passed to the macro should
be separated with a space (a comma must not be used in this case).

You now know how macros are called, but what do they do? As I have said, they
rewrite your code to generate new, transformed code. You can easily see this rewritten
code by using the @macroexpand macro. Let’s start with a simple example of the
@assert macro:

In [10]:
@macroexpand @assert(1 == 2, "1 is not equal 2")

:(if 1 == 2
      nothing
  else
      Base.throw(Base.AssertionError("1 is not equal 2"))
  end)

Of course, normally macros can generate much more complex code. For example,
the @time macro performs multiple operations to ensure proper measurement of the
execution time of the passed expression:

In [11]:
@macroexpand @time(1+2)

quote
    #= timing.jl:252 =#
    begin
        #= timing.jl:257 =#
        $(Expr(:meta, :force_compile))
        #= timing.jl:258 =#
        local var"#43#stats" = Base.gc_num()
        #= timing.jl:259 =#
        local var"#45#elapsedtime" = Base.time_ns()
        #= timing.jl:260 =#
        Base.cumulative_compile_timing(true)
        #= timing.jl:261 =#
        local var"#46#compile_elapsedtimes" = Base.cumulative_compile_time_ns()
        #= timing.jl:262 =#
        local var"#44#val" = $(Expr(:tryfinally, :(1 + 2), quote
    var"#45#elapsedtime" = Base.time_ns() - var"#45#elapsedtime"
    #= timing.jl:264 =#
    Base.cumulative_compile_timing(false)
    #= timing.jl:265 =#
    var"#46#compile_elapsedtimes" = Base.cumulative_compile_time_ns() .- var"#46#compile_elapsedtimes"
end))
        #= timing.jl:267 =#
        local var"#47#diff" = Base.GC_Diff(Base.gc_num(), var"#43#stats")
        #= timing.jl:268 =#
        local var"#48#_msg" = Base.nothing
        #= timing.jl:269 =#
 

Now you might ask why @time is a macro and not just a function. If you were to
define the time function instead and write time(1 + 2), the 1 + 2 expression would
be evaluated before it was passed to the function, so it would be impossible to measure
the time it took to execute it. To measure the execution time of an expression, we
must augment it with proper code before the expression is run. This is possible only
during parsing of the Julia code.

---
For benchmarking, we will use the @benchmark macro
from the BenchmarkTools.jl package. It differs from the @time macro in that it runs
the expression many times and then calculates the statistics of the observed run times

In [13]:
using BenchmarkTools

In [14]:
function winsorized_mean(x::AbstractVector, k::Integer)
    sorted_x = sort(x)
    sorted_x[1 : k] = sorted_x[k+1 : 2*k]
    sorted_x[ end - k + 1 : end ] = sorted_x[ (end - 2 * k + 1) : end - k ]
    # print(sorted_x)
    return sum(sorted_x)/length(sorted_x)
end


winsorized_mean (generic function with 1 method)

In [16]:
x = rand(10^6);

In [20]:
size(x)

(1000000,)

In [21]:
@benchmark winsorized_mean($x, 10^5)

BenchmarkTools.Trial: 69 samples with 1 evaluation.
 Range (min … max):  70.273 ms … 82.426 ms  ┊ GC (min … max): 0.00% … 0.69%
 Time  (median):     72.540 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   73.082 ms ±  2.111 ms  ┊ GC (mean ± σ):  0.12% ± 0.26%

        █ ▁▃ ▃                                                 
  ▄▄▁▄▄▄█▄██▇█▁▇▆▆▆▆▆▄▄▁▇▄▇▁▄▆▄▄▄▁▁▆▇▁▁▁▄▁▄▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  70.3 ms         Histogram: frequency by time        79.4 ms <

 Memory estimate: 9.16 MiB, allocs estimate: 6.

---

Now we benchmark computation of the winsorized mean by using the functions pro-
vided by the packages from the Julia statistics ecosystem:

In [22]:
using Statistics
using StatsBase

In [23]:
@benchmark mean(winsor($x; count=10^5))

BenchmarkTools.Trial: 280 samples with 1 evaluation.
 Range (min … max):  16.138 ms … 24.870 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.738 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.848 ms ±  1.392 ms  ┊ GC (mean ± σ):  0.61% ± 1.34%

    █▇█▄     ▂  ▂▂▄▄▂ ▁                                        
  ▄▇█████▅█▄██▇▆███████▃▇▄▄▅▆▄▁▃▁▃▆▁▄▁▃▁▃▃▄▃▁▃▁▃▁▁▁▁▁▁▁▃▃▁▁▁▃ ▃
  16.1 ms         Histogram: frequency by time        22.8 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [25]:
# <!-- An important aspect of using the @benchmark macro is that we use $x instead of
# just x. This is needed to get a correct assessment of execution time of the expressions
# we check. As a rule, remember to prefix with $ all global variables you use in the
# expressions you want to benchmark (this applies only to benchmarking and is not a
# general rule when using macros) -->

The short explanation is as follows. Recall that since x is a global
variable, code using it is not type stable. When the @benchmark macro sees the $x, it
is instructed to turn the x variable into one that is local (and thus type stable) before
running the benchmarks.

The BenchmarkTools.jl package also provides the @btime macro that accepts the
same arguments as @benchmark. The difference is that it produces less-verbose out-
put, similar to @time, and the printed time is the minimum elapsed time measured
during benchmarking. Here is an example:

In [26]:
@btime mean(winsor($x; count=10^5))

  16.141 ms (2 allocations: 7.63 MiB)


0.4998842942950655

@edit is one of my favorite macros. In your source code editor, it takes you directly to
the source code of the function you are using (you can specify which editor should beSummary
used by setting the JULIA_EDITOR environment variable

In [28]:
# @edit winsor(x, count=10^5)

In [36]:
x = collect(1:10^6);

In [37]:
size(x)

(1000000,)

In [38]:
@time y = sort(x)

  0.014246 seconds (56 allocations: 7.632 MiB, 26.61% compilation time)


1000000-element Vector{Int64}:
       1
       2
       3
       4
       5
       6
       7
       8
       9
      10
      11
      12
      13
       ⋮
  999989
  999990
  999991
  999992
  999993
  999994
  999995
  999996
  999997
  999998
  999999
 1000000

In [39]:
@benchmark y = sort($x)

BenchmarkTools.Trial: 538 samples with 1 evaluation.
 Range (min … max):  8.310 ms … 15.692 ms  ┊ GC (min … max): 0.00% … 6.46%
 Time  (median):     8.774 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.295 ms ±  1.192 ms  ┊ GC (mean ± σ):  1.00% ± 2.31%

   ██▇▂                                                       
  ██████▇▄▆▄▅▆▄▄▅▃▃▃▃▄▃▄▃▃▃▁▁▂▃▃▂▂▃▃▂▂▃▂▃▂▁▂▃▃▁▂▁▂▂▃▂▁▂▂▂▂▁▂ ▃
  8.31 ms        Histogram: frequency by time        13.5 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.